In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
import random
import os
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [20, 10]

tf.__version__ 

Autoenkodery mogą być używane do wielu różnych rzeczy, takich jak segmentacja obrazu, redukcja wymiarowości, wykrywanie anomalii i wiele innych. Dzisiaj zaczniemy od prostego autoenkodera odszumiającego. Użyjemy zestawu danych MNIST dostępnego w Keras:

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1) / 255
x_test = x_test.reshape(-1, 28, 28, 1) / 255

W zadaniu odszumiania wprowadzamy do autoenkodera oryginalne obrazy (danye) z dodanym szumem. Wyjście to po prostu oryginalny obraz (dane). Chcemy zminimalizować **błąd rekonstrukcji** między zaszumionym a oryginalnym obrazem. Aby to zrobić, musimy najpierw utworzyć zestaw danych pociągu zaszumionego:

In [ ]:
x_train_noise = np.clip(x_train + np.random.normal(0, 0.5, x_train.shape), 0, 1)
x_test_noise = np.clip(x_test + np.random.normal(0, 0.5, x_test.shape), 0, 1)

Teraz możemy zbudować prosty autoenkoder odszumiający:

In [ ]:
# Zainiclalizuj model
autoencoder = ___
# Encoder part
# Dodaj warstweę konwolucji 2D: 32 filtry wielkości 3x3, aktywacja relu, padding "same"
___
# Dodaj warstwę Max pooling 2D: 2x2, padding "same"
___
# Dodaj warstweę konwolucji 2D: 32 filtry wielkości 3x3, aktywacja relu, padding "same"
___
# Dodaj warstwę Max pooling 2D: 2x2, padding "same"
___
# Decoder part
# Dodaj warstweę konwolucji 2D: 32 filtry wielkości 3x3, aktywacja relu, padding "same"
___
# Dodaj warstwę UpSamplingu 2D: size 2x2
___
# Dodaj warstweę konwolucji 2D: 32 filtry wielkości 3x3, aktywacja relu, padding "same"
___
# Dodaj warstwę UpSamplingu 2D: size 2x2
___
# Dodaj warstweę konwolucji 2D: 32 filtry wielkości 3x3, aktywacja relu, padding "same"
___
autoencoder.summary()

In [ ]:
# Skompiluj model

In [ ]:
# Wytrenuj model

Zobaczmy prognozy dotyczące zestawu testowego z szumami:

In [ ]:
autoencoder_predictions = autoencoder.predict(x_test_noise)

In [ ]:
i = 9
print("Original:")
plt.matshow(x_test[i,:, :, 0])
print("Noisy:")
plt.matshow(x_test_noise[i,:, :, 0])
print("Reconstructed:")
plt.matshow(autoencoder_predictions[i,:, :, 0])

Z powyższego przykładu widzieliście, że w zadaniu oznaczającym byliśmy zainteresowani wyjściem **dekodera** (rekonstrukcja), teraz użyjemy autoenkoderów do redukcji wymiarowości. Tym razem będziemy zainteresowani wyjściem **enkodera** (reprezentacja niskowymiarowa). Po raz kolejny wykorzystamy zbiór danych MNIST. Tym razem chcemy zbudować autoenkoder oparty na MLP, więc będziemy musieli przekształcić dane w wektory:

In [ ]:
x_train_vec = x_train.reshape((60000, 784))
x_test_vec = x_test.reshape((10000, 784))

Tym razem nie możemy po prostu skorzystać z modelu sekwencyjnego, będziemy musieli skorzystać z funkcjonalnego API:

In [ ]:
# Stwórz Input do API funkcyjnego
inputs = ___

# Dodaj warstwę głęboką: 128 neuronów, aktywacja relu
encoder = ___
# Dodaj warstwę głęboką: 64 neurony, aktywacja relu
encoder = ___
# Dodaj warstwę głęboką: 32 neurony, aktywacja relu
encoder = ___

# Dodaj warstwę głęboką: 64 neurony, aktywacja relu
decoder = ___
# Dodaj warstwę głęboką: 128 neuronów, aktywacja relu
decoder = ___
# Dodaj warstwę głęboką: 784 neuronów, aktywacja relu
decoder = ___

# Stwórz model przy pomocy API funkcyjnego
autoencoder = ___
autoencoder.summary()

In [ ]:
encoder = tf.keras.Model(inputs, encoder)
encoder.summary()

Skompilujemy model z `mse` jako funkcją straty:

In [ ]:
# Skompiluj model

In [ ]:
# Wytrenuj model

Teraz możemy sprawdzić rekonstrukcje i reprezentacje niskowymiarowe:

In [ ]:
autoencoder_predictions = autoencoder.predict(x_test_vec)
encoder_predictions = encoder.predict(x_test_vec)

In [ ]:
i = 0
print("Original:")
plt.matshow(x_test_vec[i,:].reshape((28, 28)))
print("Low-dim:")
plt.matshow(encoder_predictions[i,:].reshape((1, 32)))
print("Reconstructed:")
plt.matshow(autoencoder_predictions[i,:].reshape((28, 28)))

In [ ]:
encoder_predictions[i,:].shape

W ostatnim zadaniu użyjemy autoenkoderów do wykrywania anomalii. Będziemy korzystać ze zbioru danych dotyczących oszustw kredytowych:

In [ ]:
creditcard_train_X = pd.read_csv("data/creditcard_train_X", sep=" ").to_numpy()
creditcard_test_X = pd.read_csv("data/creditcard_test_X", sep=" ").to_numpy()
creditcard_train_Y = pd.read_csv("data/creditcard_train_Y", sep=" ").to_numpy()
creditcard_test_Y = pd.read_csv("data/creditcard_test_Y", sep=" ").to_numpy()

print(creditcard_train_X.shape)
print(creditcard_train_Y.shape)

W zadaniu wykrywania anomalii interesuje nas znalezienie próbek (transakcji), które mają duży błąd rekonstrukcji, co może nam powiedzieć, że jest w nich coś niezwykłego. W tym zadaniu możemy użyć modelu sekwencyjnego podobnie jak w zadaniu odszumiającym.

In [ ]:
# Zainicjalizuj model
autoencoder = ___
# Dodaj warstwę głęboką: 14 neuronów, aktywacja tanh
___
# Dodaj warstwę głęboką: 7 neuronów, aktywacja relu
___
# Dodaj warstwę głęboką: 7 neuronów, aktywacja tanh
___
# Dodaj warstwę głęboką: 29 neuronów, aktywacja relu
___
autoencoder.summary()

In [ ]:
# Skompiluj model
___

In [ ]:
# Wytrenuj model
___

Teraz możemy obliczyć błąd rekonstrukcji zestawu testowego i znaleźć najlepszy punkt odcięcia dla oszustwa związanego z kartą kredytową:

In [ ]:
predictions = autoencoder.predict(creditcard_test_X)
reconstruction_error = ((creditcard_test_X - predictions)**2).mean(axis=1)

In [ ]:
results = pd.DataFrame({
    'reconstruction_error': reconstruction_error,
    'fraud': creditcard_test_Y[:, 0]
})

In [ ]:
results.hist(column = 'reconstruction_error', by = 'fraud')

In [ ]:
from sklearn.metrics import precision_recall_curve
precision, recall, thresholds = precision_recall_curve(results.fraud, results.reconstruction_error)
alpha = 0.004
denom = alpha/precision + (1-alpha)/recall
f_scores = np.divide(1, denom, out=np.zeros_like(denom), where=(denom!=0))
max_f = np.max(f_scores)
max_f_thresh = thresholds[np.argmax(f_scores)]

pd.crosstab(results.fraud, results.reconstruction_error >= max_f_thresh,
           rownames = ["true"], colnames = ["predicted"])

In [ ]:
max_f_thresh